In [1]:
from langchain_community.llms.ollama import Ollama

def initialize_models_query():
    ollama_llm_mistral = Ollama(
        model="mistral-nemo",
        base_url="http://192.168.180.131:11434"
    )
    ollama_llm_llama = Ollama(
        model="llama3.1:8b",
        base_url="http://192.168.180.131:11434"
    )
    return ollama_llm_mistral, ollama_llm_llama

In [2]:
def transform_query_with_ollama(query, llm):
    """
    Sendet eine Query an das LLM (Ollama), zusammen mit dem gefilterten Kontext.
    
    Parameters:
    - query: Die ursprüngliche Frage.
    - llm: Das LLM-Objekt, das die Anfrage bearbeitet.
    
    Returns:
    - Die Antwort des LLM.
    """
    prompt_text = f"""
        Sie sind ein KI-Assistent, der komplexe Anfragen in einfachere Unteranfragen für ein RAG-System zerlegt. Angesichts der ursprünglichen Anfrage zerlegen Sie diese in 1-4 einfachere Unteranfragen. Diese Unteranfragen sollen eine umfassende Antwort auf die ursprüngliche Anfrage liefern.

	•	Falls es nur eine Frage ist, geben Sie diese als einzelne Unteranfrage in Listenform aus.
	•	Falls die Anfrage unnötige Informationen für das RAG-System enthält, entfernen Sie diese.
	•	Geben Sie die Unteranfragen immer nummeriert und in Listenform aus.

    Beispiele:

    Beispiel 1:
    Ursprüngliche Anfrage: Was kommt in die gelbe Tonne und was kommt in die blaue Tonne?

    Unteranfragen:

        1.	Was kommt in die gelbe Tonne?
        2.	Was kommt in die blaue Tonne?

    Beispiel 2:
    Ursprüngliche Anfrage: Wo entsorge ich mein Altglas? Fasse dich in 100 Wörter.

    Unteranfrage:

        1.	Wo entsorge ich mein Altglas?

    Hier ist die Anfrage, die du in Unteranfragen gliedern sollst:
    Ursprüngliche Anfrage: '{query}'
        
    Gebe nur die Liste an Unteranfragen aus!    
    """
    
    response = llm.invoke(prompt_text)
    
    return response

In [4]:
def start_query_transform(query):

    mistral_model, llama_model = initialize_models_query()
    response_llm = transform_query_with_ollama(query, mistral_model)
    # print(response_llm)
    return response_llm

import re

def string_in_liste_aufteilen(text):
    # Wir verwenden eine reguläre Expression, um den String bei Zahlen und Fragezeichen aufzuteilen
    fragen = re.split(r'\d+\.\s', text)
    # Filtere mögliche leere Elemente heraus
    fragen = [frage.strip() for frage in fragen if frage]
    return fragen



# query = "Was kommt in den Altglascontainer? Gebe mir die Antwort in 100 Wörtern."
query = "Was kommt in den Altglascontainer und was in die Altkleidersammlung? Fasse dich in 100 Wörter!"
# query = "Was kommt in den Restmüll und was kommt in den Biomüll?"

query_transform_string = start_query_transform(query)

# Funktion aufrufen
liste = string_in_liste_aufteilen(query_transform_string)
print(liste)

['', 'Was kommt in den Altglascontainer?', 'Was kommt in die Altkleidersammlung?']


In [5]:
from query_simple_rag import *

def start_each_query(query):
    
    # mistral_model, llama_model = initialize_models()
    collection_name='document_embeddings_v10'
    persist_directory='chroma_db_v4'
    
    rag_results = get_similar_documents_from_chroma(collection_name, persist_directory, query)
    filter_context = create_context_for_llm(rag_results)
    
    # response_llm_mistal = ask_llm_with_ollama(query, filter_context, mistral_model)

    return filter_context

def dokumente_ausgeben(liste):
    multi_filter_context = ""
    for i, dokument in enumerate(liste, 1):  # i ist der Index, dokument ist der Inhalt
        # print(f"{i}. {dokument}")
        multi_filter_context += f"Kontext {i+1}:\n"
        get_filter_context = start_each_query(dokument)
        multi_filter_context += f"{get_filter_context}\n\n"
        
    return multi_filter_context


# Dokumente ausgeben
multi_filtert_context_result = dokumente_ausgeben(liste)



/Volumes/data1/Programming/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Embedding erstellt
Embedding erstellt
Embedding erstellt


In [6]:
from IPython.display import display, HTML

def print_corrections(query, multi_filtert_context_result):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Frage</h3>
        <p style="white-space: pre-wrap; color: white;">{query}</p>
        <h3>Antwort Mistral: </h3>
        <p style="white-space: pre-wrap; color: green;">{multi_filtert_context_result}</p>
    </div>
    """))

print_corrections(query, multi_filtert_context_result)

In [7]:
def ask_llm_with_multi_context(query, multi_filtert_context_result):
    mistral_model, llama_model = initialize_models()
    response_llm_mistal = ask_llm_with_ollama(query, multi_filtert_context_result, mistral_model)

    return response_llm_mistal

answer = ask_llm_with_multi_context(query, multi_filtert_context_result)

print(answer)

In den Altglascontainer gehören saubere Flaschen und Deckelgläser, getrennt nach Weiß, Grün, Braun und Blau/Bunt. Konservengläser können auch hinein. Aus dem Altglascontainer sollten jedoch Flachglas wie Spiegel oder Scheiben, Trinkgläser, Blumenvasen, Porzellan, Keramik, Glühbirnen, stark verschmutzte Gläser sowie Energiesparlampen und Kunststoffverpackungen wegbleiben.

In die Altkleidersammlung gehören saubere Altkleider, Schuhe und Leder. Hier sollten jedoch keine verschmutzten oder stark verunreinigten Teile hinzukommen. Auch Textilien aus nicht saugfähigen Materialien wie Gummi, PVC oder Kunstfasern sind nicht geeignet für die Altkleidersammlung.

Es ist wichtig, dass man sich an diese Trennung hält, um eine effektive Recycling-Prozess zu gewährleisten und die Lebensdauer der Container zu verlängern.
